In [33]:
from os import listdir, walk
from os.path import isfile, join
import numpy as np
from PIL import Image
import time
import cv2
from sys import getsizeof
from IPython.display import display
import random
import math
import datetime

import tensorflow as tf
from tensorflow.keras.applications.xception import Xception, decode_predictions
from tensorflow.keras.layers import Input
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
#tf.config.optimizer.set_jit(True)
#tf.debugging.set_log_device_placement(True)
print(tf.config.experimental.list_physical_devices())

%run ./variables.ipynb
%run ./utils.ipynb

id_map = get_selected_taxons()
n_classes = len(list(id_map.keys()))

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Handle data

In [38]:
x_train, y_train, x_test, y_test = get_dataset(percentage_train = 0.9)
dataset_train = DiatomSequence(x_train, y_train, 30)

'185/185'

# Model

In [39]:
# fetching base model
base_model = Xception(include_top=False, weights='imagenet', input_shape=(256, 256, 3), pooling=None)

In [40]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(185, activation='softmax')(x)
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [42]:
!rm -rf LOG_DIR
log_dir = LOG_DIR + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(dataset_train, epochs=10, use_multiprocessing=True, callbacks=[tensorboard_callback])

  ...
    to  
  ['...']


  ...
    to  
  ['...']


Train for 1110 steps
Epoch 1/10


1101/1110 [============================>.] - ETA: 2s - loss: 81989456.8668 - accuracy: 3.0293e-05WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.


1110/1110 [==============================] - 357s 322ms/step - loss: 83227824.7395 - accuracy: 3.0047e-05
Epoch 2/10


1101/1110 [============================>.] - ETA: 2s - loss: 531667397.7402 - accuracy: 0.0000e+00WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.


1110/1110 [==============================] - 354s 319ms/step - loss: 534594789.5811 - accuracy: 0.0000e+00
Epoch 3/10


1101/1110 [============================>.] - ETA: 2s - loss: 1446550179.9237 - accuracy: 0.0000e+00WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.


1110/1110 [==============================] - 355s 320ms/step - loss: 1450355402.6335 - accuracy: 0.0000e+00
Epoch 4/10


  52/1110 [>.............................] - ETA: 5:40 - loss: 2118245969.2308 - accuracy: 0.0000e+00

In [25]:
dataset_test = DiatomSequence(x_test, y_test, 1)

In [28]:
preds = model.predict(dataset_test, use_multiprocessing=True)

In [37]:
labels = [np.argmax(pred) for pred in preds]